# Doppler Solve

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
from tqdm.auto import tqdm

In [ ]:
nc = 2
map = starry.DopplerMap(15, nc=nc, veq=50000, inc=75, nt=32)

In [ ]:
# Component surface images
images = ["star", "spot"][:nc]

# Component spectra
mu = np.array([643.1, 642.9])
sig = 0.025
dw = map.wav0.reshape(1, -1) - mu.reshape(-1, 1)
spectra = (1.0 - np.exp(-0.5 * dw ** 2 / sig ** 2))[:nc]

# Load the component maps
map.load(images=images, spectra=spectra, smoothing=0.075)
y_true = np.array(map.y)

# Show
map.show_components(show_spectra=False)

In [ ]:
# Generate
flux_err = 1e-6
map.reset(veq=50000, inc=75)
map.load(images=images, spectra=spectra, smoothing=0.075)
flux = map.flux(normalize=False)
flux += flux_err * np.random.randn(*flux.shape)

# Solve
map.solve(flux, normalized=False, fix_spectrum=True, flux_err=flux_err)
map.show_components(show_spectra=False)

In [ ]:
# Generate
flux_err = 1e-6
map.reset(veq=50000, inc=75)
map.load(images=images, spectra=spectra, smoothing=0.075)
flux = map.flux(normalize=True)
flux += flux_err * np.random.randn(*flux.shape)

# Solve
map.solve(flux, normalized=True, fix_spectrum=True, flux_err=flux_err)
map.show_components(show_spectra=False)

In [ ]:
# Generate
flux_err = 1e-6
map.reset(veq=50000, inc=75)
map.load(images=images, spectra=spectra, smoothing=0.075)
flux = map.flux(normalize=True)
flux += flux_err * np.random.randn(*flux.shape)

# Solve
map.solve(flux, normalized=True, fix_spectrum=True, flux_err=flux_err, niter=100000)
map.show_components(show_spectra=False)